# Spatial Regression Formula Demo

**By Tyler D. Hoffman**

This notebook demonstrates the syntax and usage of formula notation for spatial and nonspatial regression models in `spreg`. The formula notation allows users to express their (spatial) regressions in an organic manner akin to the way they might write them down mathematically. Additionally, the formula syntax handles preprocessing natively, potentially saving many lines of code.

## Imports

In [1]:
import os
os.chdir("..")  # make sure to run the notebook in spreg/, not spreg/notebooks

In [2]:
import spreg
import numpy as np
import pandas as pd
import geopandas as gpd
from libpysal.examples import load_example
from libpysal.weights import Kernel, fill_diagonal

## Load data 

We'll use the Boston housing example for this demonstration.

In [3]:
boston = load_example("Bostonhsg")
boston_df = gpd.read_file(boston.get_path("boston.shp"))

Example not available: Bostonhsg
Example not downloaded: Chicago parcels
Example not downloaded: Chile Migration
Example not downloaded: Spirals


### Make weights matrix 

We'll use a kernel weights matrix. We also set the diagonal of the weights matrix to 0 (necessary for lag model).

In [4]:
weights = Kernel(boston_df[["x", "y"]], k=50, fixed=False)
weights = fill_diagonal(weights, 0)

## Test the formulas

The function signature is:

```python
spreg.from_formula(formula, df, w=None, method="gm", skedastic=None, debug=False, **kwargs)
```

where:

- `formula` is a string following formulaic's syntax and the below additional syntax
- `df` is a `pandas.DataFrame` or `geopandas.GeoDataFrame`
- `w` is a `libpysal.weights.W` object
- `method` is a string describing the estimation method (for spatial lag or error models)
- `skedastic` is a string specifying the form of skedasticity in an error term (if at all)
- `debug` is a boolean which (when true) outputs the parsed formula alongside the configured model
- `**kwargs` are additional arguments to be passed to the model class.

The new formula syntax comes in two parts:

- The `<...>` operator:
  - Enclose variables (covariates or response) in angle brackets to denote a spatial lag.
  - `<` and `>` are not reserved characters in `formulaic` (the underlying formula parsing library), so there are no conflicts.
  - **Usage:** include `<FIELD>` as a term in a formula string to add that field and its spatial lag field from the dataframe to model matrix.
  - Can use other transformations within `<...>`, e.g. `<{10*FIELD1} + FIELD2>` will be correctly parsed.
- The `&` symbol:
  - Adds a spatial error component to a model.
  - `&` is not a reserved character in formulaic, so there are no conflicts.
  - **Usage:** include `&` as a term in a formula string to introduce a spatial error component in a model.

The parser accepts combinations of `<...>` and `&`: `<FIELD1 + ... + FIELDN> + &` is the most general possible spatial model available. Additionally, any transformations from [`formulaic`'s grammar](https://matthewwardrop.github.io/formulaic/concepts/grammar/) are admissible anywhere in the formula string.

Variable names do not need to be provided separately to the model classes (i.e., through the `name_x` and `name_y` keyword arguments) as they can be automatically detected from the formula string. Variables which read ``w.sparse @ `FIELD` `` are the spatial lags of those fields (future TODO: make this prettier).

### Default `spreg.OLS` usage

For comparison, we'll begin with showing how one would run this model using `spreg.OLS`. These variable transformations are inspired by the original paper (Harrison and Rubinfeld, 1978).

In [5]:
boston_df["NOXSQ"] = (10 * boston_df["NOX"])**2
boston_df["RMSQ"] = boston_df["RM"]**2
boston_df["LOGDIS"] = np.log(boston_df["DIS"].values)
boston_df["LOGRAD"] = np.log(boston_df["RAD"].values)
boston_df["TRANSB"] = boston_df["B"].values / 1000
boston_df["LOGSTAT"] = np.log(boston_df["LSTAT"].values)
boston_df["LCMEDV"] = np.log(boston_df["CMEDV"].values)

fields = ["RMSQ", "AGE", "LOGDIS", "LOGRAD", "TAX", "PTRATIO",
          "TRANSB", "LOGSTAT", "CRIM", "ZN", "INDUS", "CHAS", "NOXSQ"]
X = boston_df[fields].values
y = boston_df["LCMEDV"].values  # predict log corrected median house prices from covars

In [6]:
model = spreg.OLS(y, X, name_y="LCMEDV", name_x=fields)
print(model.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :      LCMEDV                Number of Observations:         506
Mean dependent var  :      3.0346                Number of Variables   :          14
S.D. dependent var  :      0.4083                Degrees of Freedom    :         492
R-squared           :      0.8108
Adjusted R-squared  :      0.8058
Sum squared residual:      15.930                F-statistic           :    162.1449
Sigma-square        :       0.032                Prob(F-statistic)     :  2.334e-168
S.E. of regression  :       0.180                Log likelihood        :     156.980
Sigma-square ML     :       0.031                Akaike info criterion :    -285.960
S.E of regression ML:      0.1774                Schwarz criterion     :    -226.788

-----------------------------------------------------------------------------

### OLS Model with manually transformed fields

This demonstrates the basic use case of `spreg.from_formula`: to configure and run a regression model. Here, there are no spatial terms and all the fields used are directly available from the underlying dataframe. Additionally, this demonstrates how to pass keyword arguments from the `from_formula` call to underlying models: the `robust` argument is passed to the OLS model and appears in the outputted summary. 

In [7]:
formula = "LCMEDV ~ RMSQ + AGE + LOGDIS + LOGRAD + TAX + PTRATIO + TRANSB" + \
          " + LOGSTAT + CRIM + ZN + INDUS + CHAS + NOXSQ"
model = spreg.from_formula(formula, boston_df, robust="white")
print(model.summary)    

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :      LCMEDV                Number of Observations:         506
Mean dependent var  :      3.0346                Number of Variables   :          14
S.D. dependent var  :      0.4083                Degrees of Freedom    :         492
R-squared           :      0.8108
Adjusted R-squared  :      0.8058
Sum squared residual:      15.930                F-statistic           :    162.1449
Sigma-square        :       0.032                Prob(F-statistic)     :  2.334e-168
S.E. of regression  :       0.180                Log likelihood        :     156.980
Sigma-square ML     :       0.031                Akaike info criterion :    -285.960
S.E of regression ML:      0.1774                Schwarz criterion     :    -226.788

White Standard Errors
-------------------------------------------------------

### OLS model, fields transformed using formulaic

This demonstrates some of the transformations that `formulaic` can do. It's the same model as the last two examples, but all the transformations are specified using `formulaic`'s grammar.

In [8]:
formula = "log(CMEDV) ~ {RM**2} + AGE + log(DIS) + log(RAD) + TAX + PTRATIO" + \
          " + {B/1000} + log(LSTAT) + CRIM + ZN + INDUS + CHAS + {(10*NOX)**2}"
model = spreg.from_formula(formula, boston_df)
print(model.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :  log(CMEDV)                Number of Observations:         506
Mean dependent var  :      3.0346                Number of Variables   :          14
S.D. dependent var  :      0.4083                Degrees of Freedom    :         492
R-squared           :      0.8108
Adjusted R-squared  :      0.8058
Sum squared residual:      15.930                F-statistic           :    162.1449
Sigma-square        :       0.032                Prob(F-statistic)     :  2.334e-168
S.E. of regression  :       0.180                Log likelihood        :     156.980
Sigma-square ML     :       0.031                Akaike info criterion :    -285.960
S.E of regression ML:      0.1774                Schwarz criterion     :    -226.788

-----------------------------------------------------------------------------

### SLX model

Note that the outputted model is `spreg.OLS`, as SLX simply includes the covariates. This also demonstrates the usage of the `debug=True` flag: to view the resulting formula after spatial preprocessing. We'll keep this on for the rest of the demo to give a peek under the hood.

In [9]:
formula = "log(CMEDV) ~ {RM**2} + AGE + log(RAD) + TAX + PTRATIO" + \
          " + {B/1000} + log(LSTAT) + <CRIM + ZN + INDUS + CHAS> + {(10*NOX)**2}"
model, parsed_formula = spreg.from_formula(formula, boston_df, w=weights, debug=True)
print(type(model))
print(parsed_formula)
print(model.summary)

<class 'spreg.ols.OLS'>
log(CMEDV) ~ {RM**2} + AGE + log(RAD) + TAX + PTRATIO + {B/1000} + log(LSTAT) + CRIM + ZN + INDUS + CHAS + {w.sparse @ `CRIM`} + {w.sparse @ `ZN`} + {w.sparse @ `INDUS`} + {w.sparse @ `CHAS`} + {(10*NOX)**2} - 1
REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :  log(CMEDV)                Number of Observations:         506
Mean dependent var  :      3.0346                Number of Variables   :          17
S.D. dependent var  :      0.4083                Degrees of Freedom    :         489
R-squared           :      0.8136
Adjusted R-squared  :      0.8075
Sum squared residual:      15.692                F-statistic           :    133.3901
Sigma-square        :       0.032                Prob(F-statistic)     :  1.176e-166
S.E. of regression  :       0.179                Log likelihood        :     160.790
Sigma-square

### SLY model

The default estimation routine for spatial lag and error models is generalized method of moments.

In [10]:
formula = "log(CMEDV) ~ {RM**2} + AGE + <log(CMEDV)>"
model, parsed_formula = spreg.from_formula(formula, boston_df, w=weights, debug=True)
print(type(model))
print(parsed_formula)
print(model.summary)

<class 'spreg.twosls_sp.GM_Lag'>
log(CMEDV) ~ {RM**2} + AGE - 1
REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :  log(CMEDV)                Number of Observations:         506
Mean dependent var  :      3.0346                Number of Variables   :           4
S.D. dependent var  :      0.4083                Degrees of Freedom    :         502
Pseudo R-squared    :      0.5122
Spatial Pseudo R-squared:  0.5172

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       2.3761615       0.1165066      20.3950774       0.0000000
                 AGE      -0.0047504       0.0004623     -10.2755184       0.000

### Error model

In [11]:
formula = "log(CMEDV) ~ {RM**2} + AGE + TAX + PTRATIO + {B/1000}" + \
          " + log(LSTAT) + CRIM + ZN + INDUS + CHAS + {(10*NOX)**2} + &"
model, parsed_formula = spreg.from_formula(formula, boston_df, w=weights, debug=True)
print(type(model))
print(parsed_formula)
print(model.summary)

<class 'spreg.error_sp.GM_Error'>
log(CMEDV) ~ {RM**2} + AGE + TAX + PTRATIO + {B/1000} + log(LSTAT) + CRIM + ZN + INDUS + CHAS + {(10*NOX)**2} - 1
REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :  log(CMEDV)                Number of Observations:         506
Mean dependent var  :      3.0346                Number of Variables   :          12
S.D. dependent var  :      0.4083                Degrees of Freedom    :         494
Pseudo R-squared    :      0.7743

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       3.8739651       0.1446268      26.7859345       0.0000000
         (10*NOX)**2      -0

### Error model with ML estimation

In [12]:
formula = "log(CMEDV) ~ {RM**2} + AGE + TAX + PTRATIO + {B/1000}" + \
          " + log(LSTAT) + CRIM + ZN + INDUS + CHAS + {(10*NOX)**2} + &"
model, parsed_formula = spreg.from_formula(formula, boston_df, method="full",
                                           w=weights, debug=True)
print(type(model))
print(parsed_formula)
print(model.summary)

/home/tdh/anaconda3/envs/spreg-testing/lib/python3.9/site-packages/scipy/optimize/_minimize.py:863: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
/mnt/c/Users/tyler/OneDrive/Documents/research/spreg/spreg/ml_error.py:509: RuntimeWarning: invalid value encountered in log
  jacob = np.log(np.linalg.det(a))


<class 'spreg.ml_error.ML_Error'>
log(CMEDV) ~ {RM**2} + AGE + TAX + PTRATIO + {B/1000} + log(LSTAT) + CRIM + ZN + INDUS + CHAS + {(10*NOX)**2} - 1
REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :  log(CMEDV)                Number of Observations:         506
Mean dependent var  :      3.0346                Number of Variables   :          12
S.D. dependent var  :      0.4083                Degrees of Freedom    :         494
Pseudo R-squared    :      0.7654
Sigma-square ML     :       0.024                Log likelihood        :     212.183
S.E of regression   :       0.154                Akaike info criterion :    -400.366
                                                 Schwarz criterion     :    -349.647

---------------------------------------------------------------------------------

### Combo model

This model includes spatial lags of covariates, a spatial lag of the dependent variable, and a spatial error component.

In [13]:
formula = "log(CMEDV) ~ {RM**2} + AGE + TAX + {B/1000}" + \
          " + log(LSTAT) + CRIM + <ZN + CHAS + log(CMEDV)> + {(10*NOX)**2} + &"
model, parsed_formula = spreg.from_formula(formula, boston_df, w=weights, debug=True)
print(type(model))
print(parsed_formula)
print(model.summary)

<class 'spreg.error_sp.GM_Combo'>
log(CMEDV) ~ {RM**2} + AGE + TAX + {B/1000} + log(LSTAT) + CRIM + ZN + CHAS + {w.sparse @ `ZN`} + {w.sparse @ `CHAS`} + {(10*NOX)**2} - 1
REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED TWO STAGE LEAST SQUARES
-------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :  log(CMEDV)                Number of Observations:         506
Mean dependent var  :      3.0346                Number of Variables   :          13
S.D. dependent var  :      0.4083                Degrees of Freedom    :         493
Pseudo R-squared    :      0.6585
Spatial Pseudo R-squared:  0.6841

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       5.197163